In [ ]:
from transformers import GPT2Tokenizer

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Example text
long_text = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer nec odio. Praesent libero. Sed cursus ante dapibus diam. Sed nisi. Nulla quis sem at nibh elementum imperdiet. Duis sagittis ipsum. Praesent mauris. Fusce nec tellus sed augue semper porta. Mauris massa. Vestibulum lacinia arcu eget nulla. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. 

In hac habitasse platea dictumst. Curabitur sodales ligula in libero. Sed dignissim lacinia nunc. Curabitur tortor. Pellentesque nibh. Aenean quam. In scelerisque sem at dolor. Maecenas mattis. Sed convallis tristique sem. Proin ut ligula vel nunc egestas porttitor. Morbi lectus risus, iaculis vel, suscipit quis, luctus non, massa. Fusce ac turpis quis ligula lacinia aliquet. Mauris ipsum. Nulla metus metus, ullamcorper vel, tincidunt sed, euismod in, nibh. Quisque volutpat condimentum velit. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. 

Nam nec ante. Sed lacinia, urna non tincidunt mattis, tortor neque adipiscing diam, a cursus ipsum ante quis turpis. Nulla facilisi. Ut fringilla. Suspendisse potenti. Nunc feugiat mi a tellus consequat imperdiet. Vestibulum sapien. Proin quam. Etiam ultrices. Suspendisse in justo eu magna luctus suscipit. Sed lectus. 

"""

# Tokenize the text
tokenized_text = tokenizer.encode(long_text, add_special_tokens=True, max_length=512, truncation=True)



In [ ]:
txt = """
First line. 

second line

third line!
"""

enc = tokenizer.encode(txt, add_special_tokens=True, max_length=512, truncation=True)

enc

dec = tokenizer.decode(enc)

dec

In [ ]:
txtlst = [
"""first line. 

second line

third Line!""",
"new document",
"new single line"
]

# Tokenize each document separately and concatenate them with separator tokens
tokenized_documents = []
for doc in txtlst:
    tokenized_doc = tokenizer.encode(doc, add_special_tokens=False)
    tokenized_documents.extend(tokenized_doc + [tokenizer.eos_token_id])  # Adding end-of-sequence token between documents


tokenized_documents

dec = [tokenizer.decode(enc) for enc in tokenized_documents]

dec

In [ ]:
#SOTU

from nltk.corpus import state_union


In [ ]:

docs = [state_union.raw(f) for f in state_union.fileids()]

tokenized_docs = []
for doc in docs:
    tokenized_doc = tokenizer.encode(doc, add_special_tokens=False, 
                                     #max_length=1600, 
                                     #truncation=False
                                     )
    tokenized_docs.extend(tokenized_doc + [tokenizer.eos_token_id])  # Adding end-of-sequence token between documents


In [ ]:
len(tokenized_docs)

In [ ]:
tokenized_docs[:100]

dec = tokenizer.decode(tokenized_docs[:100])

dec

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

torch.cuda.empty_cache()

#import numpy as np

from transformers import GPT2Tokenizer

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


# Define your dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

# Define the Transformer model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads, hidden_size, num_layers, dropout_prob):
        super(TransformerModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.transformer_encoder_layers = TransformerEncoderLayer(embed_size, num_heads, hidden_size, dropout_prob)
        self.transformer_encoder = TransformerEncoder(self.transformer_encoder_layers, num_layers)
        self.fc = nn.Linear(embed_size, vocab_size)
        
    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        embedded = embedded.permute(1, 0, 2)  # Transformer expects sequence length first
        output = self.transformer_encoder(embedded)
        output = output.permute(1, 0, 2)  # Back to batch first
        logits = self.fc(output)
        return logits
    
# Define a function to generate text given a starting prompt
def generate_text(model, prompt, max_length=100, temperature=1.0):
    model.eval()
    with torch.no_grad():
        current_token = torch.tensor([[prompt]], dtype=torch.long)
        output_sequence = [prompt]
        
        for _ in range(max_length):
            logits = model(current_token)
            logits = logits[0, -1, :] / temperature
            probabilities = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probabilities, 1)
            output_sequence.append(next_token.item())
            current_token = next_token.unsqueeze(0)
            if next_token == 1:  # Stop generating if EOS token is generated
                break
                
    return output_sequence

def generate_text_cuda(model, prompt, max_length=100, temperature=1.0):
    model.eval()
    with torch.no_grad():
        current_token = torch.tensor([[prompt]], dtype=torch.long).cuda()
        output_sequence = [prompt]
        
        for _ in range(max_length):
            logits = model(current_token)
            logits = logits[0, -1, :] / temperature
            probabilities = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probabilities, 1)
            output_sequence.append(next_token.item())
            current_token = next_token.unsqueeze(0)
            if next_token == 1:  # Stop generating if EOS token is generated
                break
                
    return output_sequence

c:\Users\Jason\Desktop\git_control\llms\.venv\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jason\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [2]:



# Dummy data for demonstration
seq_length = 20
batch_size = 32
vocab_size = 10000
dummy_input = torch.randint(0, vocab_size, (seq_length, batch_size))  # Random input sequence

# Hyperparameters
embed_size = 128
num_heads = 4
hidden_size = 256
num_layers = 2
dropout_prob = 0.1

# Instantiate the model
model = TransformerModel(vocab_size, embed_size, num_heads, hidden_size, num_layers, dropout_prob)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training loop (dummy example)
num_epochs = 10
for epoch in range(num_epochs):
    optimizer.zero_grad()
    logits = model(dummy_input)
    loss = criterion(logits.view(-1, vocab_size), dummy_input.view(-1))
    loss.backward()
    optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Save the trained model
#torch.save(model.state_dict(), 'small_transformer_model.pth')

Epoch [1/10], Loss: 9.337352752685547
Epoch [2/10], Loss: 9.124256134033203
Epoch [3/10], Loss: 8.911956787109375
Epoch [4/10], Loss: 8.702798843383789
Epoch [5/10], Loss: 8.495234489440918
Epoch [6/10], Loss: 8.279451370239258
Epoch [7/10], Loss: 8.070791244506836
Epoch [8/10], Loss: 7.855935573577881
Epoch [9/10], Loss: 7.648670196533203
Epoch [10/10], Loss: 7.4374823570251465


In [3]:
generated_sequence = generate_text(model, prompt=10)  # You can provide any integer as the starting prompt
print('Generated sequence:', generated_sequence)


Generated sequence: [10, 2113, 2769, 3475, 6870, 4146, 1433, 8124, 1137, 6457, 6977, 4614, 113, 5238, 8457, 2596, 9320, 6566, 9481, 8181, 3738, 6848, 3120, 8719, 7896, 8487, 3210, 3692, 6693, 4061, 7374, 1765, 3991, 4094, 8489, 3938, 8033, 2718, 9513, 1168, 3036, 6718, 2074, 8283, 1550, 3787, 3688, 6725, 7719, 4979, 1211, 3354, 7915, 9649, 5164, 855, 2560, 8366, 3708, 5680, 4462, 9911, 8246, 7442, 5421, 3131, 1349, 2756, 3387, 4090, 8386, 3654, 3880, 4500, 2579, 6128, 7480, 4810, 1566, 1895, 6257, 3235, 580, 3350, 3093, 7993, 9545, 9725, 6871, 6639, 288, 2647, 1378, 2923, 1705, 4109, 295, 1354, 2907, 3536, 1058]


In [4]:
torch.cuda.is_available()

True

In [5]:
torch.cuda.empty_cache()


In [6]:
from nltk.corpus import state_union

docs = [state_union.raw(f) for f in state_union.fileids()]

tokenized_docs = []
for doc in docs:
    tokenized_doc = tokenizer.encode(doc, add_special_tokens=False, 
                                     #max_length=1600, 
                                     #truncation=False
                                     )
    tokenized_docs.extend(tokenized_doc + [tokenizer.eos_token_id])  # Adding end-of-sequence token between documents


Token indices sequence length is longer than the specified maximum sequence length for this model (2264 > 1024). Running this sequence through the model will result in indexing errors


In [7]:
input_ids = torch.tensor(tokenized_docs)

dataset = MyDataset(input_ids)

batch_size = 512

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


In [8]:
vocab_size = tokenizer.vocab_size

# Hyperparameters
num_heads = 6
embed_subsize = 32
embed_size = num_heads*embed_subsize
hidden_size = num_heads*embed_subsize
num_layers = 4
dropout_prob = 0.1


model = TransformerModel(vocab_size, embed_size, num_heads, hidden_size, num_layers, dropout_prob).cuda()



In [9]:
lr = 1e-4
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=lr)


In [13]:
# Training loop (dummy example)
num_epochs = 10
for epoch in range(num_epochs):

    total_loss = 0.0

    for batch in dataloader:

        batch=batch.cuda().unsqueeze(0)
        optimizer.zero_grad()
        logits = model(batch)
        loss = criterion(logits.view(-1, vocab_size), batch.view(-1))
        loss.backward()
        optimizer.step()

        total_loss+=loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss}')




Epoch [1/10], Loss: 52.50906245177612
Epoch [2/10], Loss: 36.409168595215306
Epoch [3/10], Loss: 24.076013173209503
Epoch [4/10], Loss: 14.805838204803877
Epoch [5/10], Loss: 8.428250377532095
Epoch [6/10], Loss: 4.603353999147657
Epoch [7/10], Loss: 2.5612542165326886
Epoch [8/10], Loss: 1.5034951445122715
Epoch [9/10], Loss: 0.9270753085438628
Epoch [10/10], Loss: 0.5903936814065673


In [14]:
generated_sequence = generate_text_cuda(model, prompt=16000,temperature=1)  # You can provide any integer as the starting prompt
print('Generated sequence:', generated_sequence)

Generated sequence: [16000, 16000, 16000, 4768, 10027, 28783, 12394, 12394, 12394, 12394, 12394, 10510, 12795, 4849, 31651, 48517, 21345, 40862, 37440, 5331, 14068, 14494, 2077, 22195, 13637, 2738, 24872, 5327, 18073, 18794, 6114, 226, 3723, 5960, 10946, 30228, 28517, 47886, 45333, 5199, 34130, 19166, 20874, 11684, 20649, 28275, 3849, 7558, 42569, 41381, 8413, 12800, 5611, 11204, 996, 2903, 10342, 15597, 25535, 27270, 10520, 17544, 30904, 4058, 4058, 4058, 4058, 4058, 4058, 4058, 20029, 36835, 24515, 15737, 2676, 16069, 48542, 20712, 26038, 22100, 14271, 14271, 14271, 24597, 1214, 8064, 26063, 11690, 14085, 40134, 3634, 8885, 9123, 11626, 1734, 35010, 6651, 46925, 9759, 20022, 13318]


In [15]:

tokenizer.decode(generated_sequence)

' Message Message Message Louisition heraldINTINTINTINTINT consciousnessclaimed Sal envy democratically Treaty heirsperformingkins treasure lesser taken Senators Rogeritution Chemical clin sponsors dent Vict�amingDes theoriesPersonal harmed melancholy marines Robert appalling clips Sugar towns decentral toutedinter constantly sponsoring fadesRem sends launched Being thoughanced searchingKeep thankfulNeitherilation650 inefficient ahead ahead ahead ahead ahead ahead aheadFive epochafort dealersiter rangeswark jungle arbitration fade ammunition ammunition ammunitionBalross ratio 224incoln questioning loafhest enorm scholars tearene simplisticcare deline referring triggers altogether'

In [16]:

total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Total number of parameters: 20464465


In [23]:
import pynvml
import time

In [24]:

def get_gpu_temperature():
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)  # Assuming you have only one GPU
    gpu_temperature = pynvml.nvmlDeviceGetTemperature(handle, pynvml.NVML_TEMPERATURE_GPU)
    pynvml.nvmlShutdown()
    return gpu_temperature


In [25]:

temperature = get_gpu_temperature()
print(f"GPU Temperature: {temperature} degrees Celsius")

GPU Temperature: 49 degrees Celsius


In [26]:
temperature

49

In [28]:

epoch = 0

while epoch < num_epochs:

    temperature = get_gpu_temperature()

    if temperature < 75:

        total_loss = 0.0

        for batch in dataloader:

            batch=batch.cuda().unsqueeze(0)
            optimizer.zero_grad()
            logits = model(batch)
            loss = criterion(logits.view(-1, vocab_size), batch.view(-1))
            loss.backward()
            optimizer.step()

            total_loss+=loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss}')
        epoch+=1
    
    else:
        print(f"Sleeping...{temperature}")
        time.sleep(30)

    

Super 1 Epoch [1/10], Loss: 0.0070765627003765985
Super 1 Epoch [2/10], Loss: 0.0048304723931096305
Super 1 Epoch [3/10], Loss: 0.0033119765279252533
Sleeping...75
